In [ ]:
# pip install numpy opencv-python

In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os
import pandas as pd
import numpy as np
import pandas as pd
import os
from PIL import Image
import torchvision.transforms as transforms
import cv2



c:\Users\hkna0\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the annotation file
annotations = pd.read_csv("../classification_annotation/anno_classi.csv")

# Example structure
print(annotations.head())  # Ensure the file is read correctly

     filename       class  truncated  occluded  observation angle    xmin  \
0  000000.txt  Pedestrian        0.0         0              -0.20  712.40   
1  000001.txt       Truck        0.0         0              -1.57  599.41   
2  000001.txt         Car        0.0         0               1.85  387.63   
3  000001.txt     Cyclist        0.0         3              -1.65  676.60   
4  000002.txt        Misc        0.0         0              -1.82  804.79   

     ymin    xmax    ymax  height  width  length   xloc  yloc   zloc  rot_y  
0  143.00  810.73  307.92    1.89   0.48    1.20   1.84  1.47   8.41   0.01  
1  156.40  629.75  189.25    2.85   2.63   12.34   0.47  1.49  69.44  -1.56  
2  181.54  423.81  203.12    1.67   1.87    3.69 -16.53  2.39  58.49   1.57  
3  163.95  688.98  193.93    1.86   0.60    2.02   4.59  1.32  45.84  -1.55  
4  167.34  995.43  327.94    1.63   1.48    2.37   3.23  1.59   8.55  -1.47  


In [4]:
# Load pre-trained ResNet model
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # Remove the last classification layer
resnet.eval()  # Set model to evaluation mode

print("ResNet50 loaded for feature extraction.")

c:\Users\hkna0\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hkna0\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet50 loaded for feature extraction.


In [5]:
# Transform for image input
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths
image_folder = "../original_data/train_images"
annotations_file = "../classification_annotation/anno_classi.csv"  # Update with your annotation file path
features_output_file = "../classification_annotation/features.csv"
labels_output_file = "../classification_annotation/labels.csv"

# SKIP THIS STEP IF YOU ALREADY GOT THE FEATURE EXTRACTED

In [6]:
# # Load annotations
# annotations = pd.read_csv(annotations_file)

# # Replace .txt with .png in the filename column
# annotations["filename"] = annotations["filename"].str.replace(".txt", ".png", regex=False)

# # Initialize storage for features and labels
# features = []
# labels = []

# # Process each annotation
# for idx, row in annotations.iterrows():
#     image_path = os.path.join(image_folder, row["filename"])
    
#     if os.path.exists(image_path):
#         try:
#             # Load and process the image
#             image = Image.open(image_path).convert("RGB")
#             xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
#             # Crop to bounding box
#             cropped_image = image.crop((xmin, ymin, xmax, ymax))
#             cropped_image = transform(cropped_image).unsqueeze(0)  # Add batch dimension

#             # Extract features using ResNet
#             with torch.no_grad():
#                 feature = resnet(cropped_image).squeeze().numpy()
            
#             # Append features and label
#             features.append(feature)
#             labels.append(row["class"])  # Store the class label
#             print(f"Processed image {row['filename']} with class {row['class']}")
#         except Exception as e:
#             print(f"Error processing image {row['filename']}: {e}")
#     else:
#         print(f"Image not found: {image_path}")

# # Save features and labels to CSV files
# features = np.array(features)
# labels = np.array(labels)

# # Save features and labels to CSV files
# np.savetxt(features_output_file, features, delimiter=",")
# np.savetxt(labels_output_file, labels, fmt="%s")

# print(f"Feature extraction complete. Saved {len(features)} features and {len(labels)} labels.")


# BUILDING MODEL

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [8]:

# Load features and labels
features = np.loadtxt("../classification_annotation/features.csv", delimiter=",")
labels = np.loadtxt("../classification_annotation/labels.csv", dtype=str)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


                precision    recall  f1-score   support

           Car       0.88      0.99      0.93      5661
       Cyclist       0.96      0.47      0.63       333
          Misc       1.00      0.31      0.47       208
    Pedestrian       0.85      0.96      0.91       934
Person_sitting       1.00      0.50      0.67        40
          Tram       0.99      0.80      0.89       102
         Truck       0.98      0.59      0.74       230
           Van       0.93      0.31      0.47       606

      accuracy                           0.88      8114
     macro avg       0.95      0.62      0.71      8114
  weighted avg       0.89      0.88      0.86      8114



In [9]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import pickle

In [10]:

# Load extracted features and labels
features = np.loadtxt(features_output_file, delimiter=",")
labels = np.loadtxt(labels_output_file, dtype=str)

# Encode labels as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# Save the LabelEncoder
with open("../scaler/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Save class names to a .npy file
np.save("../scaler/class_names.npy", label_encoder.classes_)
# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)



In [11]:
# Define a simple neural network model for classification
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
input_dim = X_train.shape[1]  # Number of features (from ResNet output)
num_classes = len(label_encoder.classes_)
model = SimpleNN(input_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_preds += (preds == labels).sum().item()
        total_preds += labels.size(0)

    # Validation step
    model.eval()
    val_loss = 0.0
    val_correct_preds = 0
    val_total_preds = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct_preds += (preds == labels).sum().item()
            val_total_preds += labels.size(0)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {correct_preds/total_preds:.4f}")
    print(f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_correct_preds/val_total_preds:.4f}")


Epoch 1/10
Train Loss: 0.4747, Train Accuracy: 0.8503
Val Loss: 0.3617, Val Accuracy: 0.8749
Epoch 2/10
Train Loss: 0.3503, Train Accuracy: 0.8837
Val Loss: 0.3047, Val Accuracy: 0.8950
Epoch 3/10
Train Loss: 0.3163, Train Accuracy: 0.8955
Val Loss: 0.2674, Val Accuracy: 0.9103
Epoch 4/10
Train Loss: 0.2956, Train Accuracy: 0.8987
Val Loss: 0.2700, Val Accuracy: 0.9095
Epoch 5/10
Train Loss: 0.2778, Train Accuracy: 0.9055
Val Loss: 0.2503, Val Accuracy: 0.9125
Epoch 6/10
Train Loss: 0.2657, Train Accuracy: 0.9100
Val Loss: 0.2396, Val Accuracy: 0.9184
Epoch 7/10
Train Loss: 0.2522, Train Accuracy: 0.9139
Val Loss: 0.2379, Val Accuracy: 0.9152
Epoch 8/10
Train Loss: 0.2486, Train Accuracy: 0.9133
Val Loss: 0.2283, Val Accuracy: 0.9205
Epoch 9/10
Train Loss: 0.2354, Train Accuracy: 0.9191
Val Loss: 0.2031, Val Accuracy: 0.9286
Epoch 10/10
Train Loss: 0.2248, Train Accuracy: 0.9223
Val Loss: 0.2211, Val Accuracy: 0.9231


In [12]:
# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
val_loss = 0.0
correct_preds = 0
total_preds = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_preds += (preds == labels).sum().item()
        total_preds += labels.size(0)

print(f"Validation Loss: {val_loss / len(val_loader):.4f}")
print(f"Validation Accuracy: {correct_preds / total_preds:.4f}")

Validation Loss: 0.2211
Validation Accuracy: 0.9231


In [14]:
# Save model state_dict
torch.save(model.state_dict(), "../model/simple_nn_classi.pth")

# Resnet

In [21]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        # Ensure that the features and labels are numpy arrays (2D for features, 1D for labels)
        self.features = np.array(features)
        self.labels = np.array(labels).flatten()  # Make sure the labels are 1D

        # Encode labels 
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

        self.transform = transform
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        if self.transform:
            feature = self.transform(feature)
        return feature, label

features = np.loadtxt("../classification_annotation/features.csv", delimiter=",")
labels = np.loadtxt("../classification_annotation/labels.csv", dtype=str)

dataset = CustomDataset(features, labels)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
# Check if the dataset is loading correctly
for inputs, labels in train_loader:
    print(inputs.shape)  # Should print something like [batch_size, feature_size]
    break


torch.Size([64, 2048])


In [25]:

class CustomResNetModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CustomResNetModel, self).__init__()
        # Fully connected layer to match the number of input features
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
input_size = 2048  
num_classes = len(set(dataset.labels))  # Number of unique classes

# Initialize the model
model = CustomResNetModel(input_size, num_classes)


In [27]:
import torch.optim as optim
import torch.nn.functional as F

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, preds = torch.max(outputs, 1)
        correct_preds += (preds == labels).sum().item()
        total_preds += labels.size(0)

    # Calculate the accuracy and loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_preds / total_preds

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), "../model/resnetmodel.pth")


Epoch 1/10
Train Loss: 0.0668, Train Accuracy: 0.9761
Epoch 2/10
Train Loss: 0.0579, Train Accuracy: 0.9797
Epoch 3/10
Train Loss: 0.0538, Train Accuracy: 0.9812
Epoch 4/10
Train Loss: 0.0415, Train Accuracy: 0.9861
Epoch 5/10
Train Loss: 0.0377, Train Accuracy: 0.9864
Epoch 6/10
Train Loss: 0.0354, Train Accuracy: 0.9876
Epoch 7/10
Train Loss: 0.0355, Train Accuracy: 0.9884
Epoch 8/10
Train Loss: 0.0273, Train Accuracy: 0.9909
Epoch 9/10
Train Loss: 0.0336, Train Accuracy: 0.9871
Epoch 10/10
Train Loss: 0.0301, Train Accuracy: 0.9893
